In [1]:
#!/usr/bin/env python
# Copyright (c) Facebook, Inc. and its affiliates.
"""
A main training script.

This scripts reads a given config file and runs the training or evaluation.
It is an entry point that is made to train standard models in detectron2.

In order to let one script support training of many models,
this script contains logic that are specific to these built-in models and therefore
may not be suitable for your own project.
For example, your research project perhaps only needs a single "evaluator".

Therefore, we recommend you to use detectron2 as an library and take
this file as an example of how to use the library.
You may want to write your own script with your datasets and other customizations.
"""
import sys
import os
# sys.path.insert(0, os.path.join(os.path.dirname(__file__), ".."))
import sys
sys.path.insert(0, '../.')
sys.path.insert(0, '../..')
sys.path.insert(0, './tools')

import logging
from collections import OrderedDict
import torch

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.modeling import build_model

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch.multiprocessing
import torch.nn.functional as F
torch.multiprocessing.set_sharing_strategy('file_system')


import lib.data.fewshot
import lib.data.ovdshot
from lib.categories import SEEN_CLS_DICT, ALL_CLS_DICT

from collections import defaultdict

import numpy as np

from detectron2.evaluation.evaluator import DatasetEvaluator
# from detectron2.data.datasets.coco_zeroshot_categories import COCO_SEEN_CLS, \
    # COCO_UNSEEN_CLS, COCO_OVD_ALL_CLS
    
from sklearn.metrics import precision_recall_curve
from sklearn import metrics as sk_metrics



class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []

        if 'OpenSet' in cfg.MODEL.META_ARCHITECTURE:
            if 'lvis' in dataset_name:
                evaluator_list.append(LVISEvaluator(dataset_name, output_dir=output_folder))
            else:
                dtrain_name = cfg.DATASETS.TRAIN[0]
                seen_cnames = SEEN_CLS_DICT[dtrain_name]
                all_cnames = ALL_CLS_DICT[dtrain_name]
                unseen_cnames = [c for c in all_cnames if c not in seen_cnames]
                evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder, few_shot_mode=True,
                                                    seen_cnames=seen_cnames, unseen_cnames=unseen_cnames, all_cnames=all_cnames))
            return DatasetEvaluators(evaluator_list)
    

        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
            evaluator_list.append(
                SemSegEvaluator(
                    dataset_name,
                    distributed=True,
                    output_dir=output_folder,
                )
            )
        if evaluator_type in ["coco", "coco_panoptic_seg"]:
            evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
        if evaluator_type == "coco_panoptic_seg":
            evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
        if evaluator_type == "cityscapes_instance":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesInstanceEvaluator(dataset_name)
        if evaluator_type == "cityscapes_sem_seg":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesSemSegEvaluator(dataset_name)
        elif evaluator_type == "pascal_voc":
            return PascalVOCDetectionEvaluator(dataset_name)
        elif evaluator_type == "lvis":
            return LVISEvaluator(dataset_name, output_dir=output_folder)
        
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        elif len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res
    

    @classmethod
    def build_model(cls, cfg):
        """
        Returns:
            torch.nn.Module:

        It now calls :func:`detectron2.modeling.build_model`.
        Overwrite it if you'd like a different model.
        """
        model = build_model(cfg)
        logger = logging.getLogger(__name__)

        # unfreeze ALL LoRA parameters
        for name, param in model.named_parameters():
            if 'lora' in name:
                param.requires_grad = True
                
        return model



def main(args):
    cfg = setup(args)
    print("CONFIG HERE:", list(cfg.keys()), '\n\n\n', list(cfg.values()))

    if args.eval_only:
        model = Trainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        if cfg.TEST.AUG.ENABLED:
            res.update(Trainer.test_with_TTA(cfg, model))
        if comm.is_main_process():
            verify_results(cfg, res)
        return res

    """
    If you'd like to do anything fancier than the standard training logic,
    consider writing your own training loop (see plain_train_net.py) or
    subclassing the trainer.
    """
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    if cfg.TEST.AUG.ENABLED:
        trainer.register_hooks(
            [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
        )
    return trainer.train()



# if __name__ == "__main__":
#     args = default_argument_parser().parse_args()
#     print("Command Line Args:", args)
#     launch(
#         main,
#         args.num_gpus,
#         num_machines=args.num_machines,
#         machine_rank=args.machine_rank,
#         dist_url=args.dist_url,
#         args=(args,),
#     )


# keys = ['VERSION', 'MODEL', 'INPUT', 'AUG', 'DATASETS', 'DATALOADER', 'SOLVER', 'INPUT_DIR', 'TEST', 'OUTPUT_DIR', 'SEED', 'CUDNN_BENCHMARK', 'VIS_PERIOD', 'GLOBAL', 'DE']
# v = [2, CfgNode({'LOAD_PROPOSALS': False, 'MASK_ON': False, 'KEYPOINT_ON': False, 'DEVICE': 'cuda', 'META_ARCHITECTURE': 'OpenSetDetectorWithExamples', 'WEIGHTS': 'weights/initial/open-vocabulary/vits+rpn.pth', 'PIXEL_MEAN': [0.48145466, 0.4578275, 0.40821073], 'PIXEL_STD': [0.26862954, 0.26130258, 0.27577711], 'BACKBONE': CfgNode({'NAME': 'build_dino_v2_vit', 'TYPE': 'small', 'FREEZE_AT': 2}), 'FPN': CfgNode({'IN_FEATURES': [], 'OUT_CHANNELS': 256, 'NORM': '', 'FUSE_TYPE': 'sum'}), 'PROPOSAL_GENERATOR': CfgNode({'NAME': 'RPN', 'MIN_SIZE': 0}), 'ANCHOR_GENERATOR': CfgNode({'NAME': 'DefaultAnchorGenerator', 'SIZES': [[32, 64, 128, 256, 512]], 'ASPECT_RATIOS': [[0.5, 1.0, 2.0]], 'ANGLES': [[-90, 0, 90]], 'OFFSET': 0.0}), 'RPN': CfgNode({'HEAD_NAME': 'StandardRPNHead', 'IN_FEATURES': ['res4'], 'BOUNDARY_THRESH': -1, 'IOU_THRESHOLDS': [0.3, 0.7], 'IOU_LABELS': [0, -1, 1], 'BATCH_SIZE_PER_IMAGE': 256, 'POSITIVE_FRACTION': 0.5, 'BBOX_REG_LOSS_TYPE': 'smooth_l1', 'BBOX_REG_LOSS_WEIGHT': 1.0, 'BBOX_REG_WEIGHTS': (1.0, 1.0, 1.0, 1.0), 'SMOOTH_L1_BETA': 0.0, 'LOSS_WEIGHT': 1.0, 'PRE_NMS_TOPK_TRAIN': 12000, 'PRE_NMS_TOPK_TEST': 6000, 'POST_NMS_TOPK_TRAIN': 2000, 'POST_NMS_TOPK_TEST': 1000, 'NMS_THRESH': 0.7, 'CONV_DIMS': [-1]}), 'ROI_HEADS': CfgNode({'NAME': 'Res5ROIHeads', 'NUM_CLASSES': 80, 'IN_FEATURES': ['res4'], 'IOU_THRESHOLDS': [0.5], 'IOU_LABELS': [0, 1], 'BATCH_SIZE_PER_IMAGE': 512, 'POSITIVE_FRACTION': 0.25, 'SCORE_THRESH_TEST': 0.001, 'NMS_THRESH_TEST': 0.5, 'PROPOSAL_APPEND_GT': True, 'SOFT_NMS_ENABLED': False, 'SOFT_NMS_METHOD': 'gaussian', 'SOFT_NMS_SIGMA': 0.5, 'SOFT_NMS_PRUNE': 0.001}), 'ROI_BOX_HEAD': CfgNode({'NAME': '', 'BBOX_REG_LOSS_TYPE': 'smooth_l1', 'BBOX_REG_LOSS_WEIGHT': 1.0, 'BBOX_REG_WEIGHTS': (10.0, 10.0, 5.0, 5.0), 'SMOOTH_L1_BETA': 0.0, 'POOLER_RESOLUTION': 7, 'POOLER_SAMPLING_RATIO': 0, 'POOLER_TYPE': 'ROIAlignV2', 'NUM_FC': 0, 'FC_DIM': 1024, 'NUM_CONV': 0, 'CONV_DIM': 256, 'NORM': '', 'CLS_AGNOSTIC_BBOX_REG': True, 'TRAIN_ON_PRED_BOXES': False}), 'ROI_BOX_CASCADE_HEAD': CfgNode({'BBOX_REG_WEIGHTS': ((10.0, 10.0, 5.0, 5.0), (20.0, 20.0, 10.0, 10.0), (30.0, 30.0, 15.0, 15.0)), 'IOUS': (0.5, 0.6, 0.7)}), 'ROI_MASK_HEAD': CfgNode({'NAME': 'MaskRCNNConvUpsampleHead', 'POOLER_RESOLUTION': 14, 'POOLER_SAMPLING_RATIO': 0, 'NUM_CONV': 0, 'CONV_DIM': 256, 'NORM': '', 'CLS_AGNOSTIC_MASK': False, 'POOLER_TYPE': 'ROIAlignV2'}), 'ROI_KEYPOINT_HEAD': CfgNode({'NAME': 'KRCNNConvDeconvUpsampleHead', 'POOLER_RESOLUTION': 14, 'POOLER_SAMPLING_RATIO': 0, 'CONV_DIMS': (512, 512, 512, 512, 512, 512, 512, 512), 'NUM_KEYPOINTS': 17, 'MIN_KEYPOINTS_PER_IMAGE': 1, 'NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS': True, 'LOSS_WEIGHT': 1.0, 'POOLER_TYPE': 'ROIAlignV2'}), 'SEM_SEG_HEAD': CfgNode({'NAME': 'SemSegFPNHead', 'IN_FEATURES': ['p2', 'p3', 'p4', 'p5'], 'IGNORE_VALUE': 255, 'NUM_CLASSES': 54, 'CONVS_DIM': 128, 'COMMON_STRIDE': 4, 'NORM': 'GN', 'LOSS_WEIGHT': 1.0}), 'PANOPTIC_FPN': CfgNode({'INSTANCE_LOSS_WEIGHT': 1.0, 'COMBINE': CfgNode({'ENABLED': True, 'OVERLAP_THRESH': 0.5, 'STUFF_AREA_LIMIT': 4096, 'INSTANCES_CONFIDENCE_THRESH': 0.5})}), 'RETINANET': CfgNode({'NUM_CLASSES': 80, 'IN_FEATURES': ['p3', 'p4', 'p5', 'p6', 'p7'], 'NUM_CONVS': 4, 'IOU_THRESHOLDS': [0.4, 0.5], 'IOU_LABELS': [0, -1, 1], 'PRIOR_PROB': 0.01, 'SCORE_THRESH_TEST': 0.05, 'TOPK_CANDIDATES_TEST': 1000, 'NMS_THRESH_TEST': 0.5, 'BBOX_REG_WEIGHTS': (1.0, 1.0, 1.0, 1.0), 'FOCAL_LOSS_GAMMA': 2.0, 'FOCAL_LOSS_ALPHA': 0.25, 'SMOOTH_L1_LOSS_BETA': 0.1, 'BBOX_REG_LOSS_TYPE': 'smooth_l1', 'NORM': ''}), 'RESNETS': CfgNode({'DEPTH': 50, 'OUT_FEATURES': ['res4'], 'NUM_GROUPS': 1, 'NORM': 'FrozenBN', 'WIDTH_PER_GROUP': 64, 'STRIDE_IN_1X1': True, 'RES5_DILATION': 1, 'RES2_OUT_CHANNELS': 256, 'STEM_OUT_CHANNELS': 64, 'DEFORM_ON_PER_STAGE': [False, False, False, False], 'DEFORM_MODULATED': False, 'DEFORM_NUM_GROUPS': 1}), 'CLIP': CfgNode({'CROP_REGION_TYPE': '', 'BB_RPN_WEIGHTS': None, 'IMS_PER_BATCH_TEST': 8, 'USE_TEXT_EMB_CLASSIFIER': False, 'TEXT_EMB_PATH': None, 'OFFLINE_RPN_CONFIG': None, 'NO_BOX_DELTA': False, 'BG_CLS_LOSS_WEIGHT': None, 'ONLY_SAMPLE_FG_PROPOSALS': False, 'MULTIPLY_RPN_SCORE': False, 'VIS': False, 'OPENSET_TEST_NUM_CLASSES': None, 'OPENSET_TEST_TEXT_EMB_PATH': None, 'CLSS_TEMP': 0.01, 'RUN_CVPR_OVR': False, 'FOCAL_SCALED_LOSS': None, 'OFFLINE_RPN_NMS_THRESH': None, 'OFFLINE_RPN_POST_NMS_TOPK_TEST': None, 'PRETRAIN_IMG_TXT_LEVEL': True, 'PRETRAIN_ONLY_EOT': False, 'PRETRAIN_RPN_REGIONS': None, 'PRETRAIN_SAMPLE_REGIONS': None, 'GATHER_GPUS': False, 'GRID_REGIONS': False, 'CONCEPT_POOL_EMB': None, 'CONCEPT_THRES': None, 'OFFLINE_RPN_LSJ_PRETRAINED': False, 'TEACHER_RESNETS_DEPTH': 50, 'TEACHER_CONCEPT_POOL_EMB': None, 'TEACHER_POOLER_RESOLUTION': 14, 'TEXT_EMB_DIM': 1024, 'GET_CONCEPT_EMB': False})}), CfgNode({'MIN_SIZE_TRAIN': (640, 672, 704, 736, 768, 800), 'MIN_SIZE_TRAIN_SAMPLING': 'choice', 'MAX_SIZE_TRAIN': 1333, 'MIN_SIZE_TEST': 800, 'MAX_SIZE_TEST': 1333, 'RANDOM_FLIP': 'horizontal', 'CROP': CfgNode({'ENABLED': False, 'TYPE': 'relative_range', 'SIZE': [0.9, 0.9]}), 'FORMAT': 'RGB', 'MASK_FORMAT': 'polygon', 'TEXT_TOKENIZER': 'openai_bpe'}), CfgNode({'SCALE': (0.08, 1.0), 'RATIO': (0.75, 1.3333333333333333), 'COLOR_JITTER': [0.4, 0.4, 0.4, 0.1, 0.0], 'GRAY_SCALE': 0.0, 'GAUSSIAN_BLUR': 0.0, 'DROPBLOCK_LAYERS': [3, 4], 'DROPBLOCK_KEEP_PROB': 1.0, 'DROPBLOCK_BLOCK_SIZE': 7, 'MIXUP_PROB': 0.0, 'MIXUP': 0.0, 'MIXCUT': 0.0, 'MIXCUT_MINMAX': [], 'MIXUP_SWITCH_PROB': 0.5, 'MIXUP_MODE': 'batch', 'MIXCUT_AND_MIXUP': False, 'INTERPOLATION': 3, 'USE_TIMM': False, 'TIMM_AUG': CfgNode({'USE_LOADER': False, 'USE_TRANSFORM': False}), 'TRAIN': CfgNode({'IMAGE_SIZE': [224, 224], 'MAX_SIZE': None}), 'TEST': CfgNode({'IMAGE_SIZE': [224, 224], 'MAX_SIZE': None, 'CENTER_CROP': False, 'INTERPOLATION': 3})}), CfgNode({'TRAIN': ('coco_2017_ovd_b_train',), 'PROPOSAL_FILES_TRAIN': (), 'PRECOMPUTED_PROPOSAL_TOPK_TRAIN': 2000, 'TEST': ('coco_2017_ovd_all_test',), 'PROPOSAL_FILES_TEST': (), 'PRECOMPUTED_PROPOSAL_TOPK_TEST': 1000, 'FACTORY_TRAIN': (), 'PATH_TRAIN': (), 'AUX': (), 'FACTORY_AUX': (), 'PATH_AUX': (), 'FACTORY_TEST': (), 'PATH_TEST': (), 'LABELMAP_FILE': '', 'ATTR_LABELMAP_FILE': '', 'FILTERED_CLASSIFICATION_DATASETS': '', 'HIERARCHY_FILE': '', 'BOX_EXTRA_FIELDS': (), 'NUM_CLASSES': 0, 'ROOT': '', 'TRAIN_SET': 'train', 'VAL_SET': '', 'TEST_SET': 'val', 'MAX_SEQ_LENGTH': 35}), CfgNode({'NUM_WORKERS': 4, 'ASPECT_RATIO_GROUPING': True, 'SAMPLER_TRAIN': 'TrainingSampler', 'REPEAT_THRESHOLD': 0.0, 'FILTER_EMPTY_ANNOTATIONS': True}), CfgNode({'LR_SCHEDULER_NAME': 'WarmupMultiStepLR', 'MAX_ITER': 90000, 'BASE_LR': 0.002, 'MOMENTUM': 0.9, 'NESTEROV': False, 'WEIGHT_DECAY': 0.0001, 'WEIGHT_DECAY_NORM': 0.0, 'GAMMA': 0.1, 'STEPS': (60000, 80000), 'WARMUP_FACTOR': 0.001, 'WARMUP_ITERS': 5000, 'WARMUP_METHOD': 'linear', 'CHECKPOINT_PERIOD': 5000, 'IMS_PER_BATCH': 16, 'REFERENCE_WORLD_SIZE': 0, 'BIAS_LR_FACTOR': 1.0, 'WEIGHT_DECAY_BIAS': 0.0001, 'CLIP_GRADIENTS': CfgNode({'ENABLED': False, 'CLIP_TYPE': 'value', 'CLIP_VALUE': 1.0, 'NORM_TYPE': 2.0}), 'AMP': CfgNode({'ENABLED': False})}), './datasets/custom_images', CfgNode({'EXPECTED_RESULTS': [], 'EVAL_PERIOD': 5000, 'KEYPOINT_OKS_SIGMAS': [], 'DETECTIONS_PER_IMAGE': 100, 'AUG': CfgNode({'ENABLED': False, 'MIN_SIZES': (400, 500, 600, 700, 800, 900, 1000, 1100, 1200), 'MAX_SIZE': 4000, 'FLIP': True}), 'PRECISE_BN': CfgNode({'ENABLED': False, 'NUM_ITER': 200})}), 'output/train/open-vocabulary/coco/vits/', -1, False, 0, CfgNode({'HACK': 1.0}), CfgNode({'CLASS_PROTOTYPES': 'weights/initial/open-vocabulary/prototypes/coco/class_prototypes_all.vitl14.pth', 'BG_PROTOTYPES': 'weights/initial/background/background_prototypes.vits14.pth', 'TOPK': 10, 'T': 128, 'RCNN_BATCH_SIZE': 128, 'POS_RATIO': 0.25, 'TEMP': 0.1, 'NUM_CLS_LAYERS': 3, 'ONE_SHOT_MODE': False, 'ONE_SHOT_REFERENCE': '', 'ONLY_TRAIN_MASK': True, 'OUT_INDICES': None, 'BG_CLS_LOSS_WEIGHT': 0.2, 'MULTIPLY_RPN_SCORE': False, 'OFFLINE_RPN_CONFIG': 'configs/RPN/mask_rcnn_R_50_C4_1x_ovd_FSD.yaml', 'OFFLINE_RPN_LSJ_PRETRAINED': False, 'OFFLINE_RPN_NMS_THRESH': None, 'OFFLINE_RPN_POST_NMS_TOPK_TEST': None})]


A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
xFormers not available


In [2]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # import ipdb; ipdb.set_trace()
    cfg.merge_from_file(args['config_file'])
    # Update cfg with new parameters
    cfg.MODEL.WEIGHTS = args['MODEL.WEIGHTS']
    cfg.OUTPUT_DIR = args['OUTPUT_DIR']
    cfg.DE.OFFLINE_RPN_CONFIG = args['DE.OFFLINE_RPN_CONFIG']
    cfg.MODEL.ADD_LORA = args.get('MODEL.ADD_LORA', False)
    cfg.freeze()
    default_setup(cfg, args)
    print(cfg.DATASETS.TEST)
    return cfg

vit = 's'

args = {
'config_file': f'configs/open-vocabulary/coco/vit{vit}.yaml',
'num_gpus': str(0),
'MODEL.WEIGHTS': '', # f'weights/initial/open-vocabulary/vit{vit}+rpn.pth'
'DE.OFFLINE_RPN_CONFIG': 'configs/RPN/mask_rcnn_R_50_C4_1x_ovd_FSD.yaml',
'OUTPUT_DIR': f'output/train/open-vocabulary/coco/vit{vit}/',
'MODEL.ADD_LORA': True,
'resume': False
}

import subprocess

# Define your variables
num_gpus = 0 
vit = 's'

from omegaconf import OmegaConf
cfg = OmegaConf.create(args)
cfg = setup(args)
cfg


[11/22 15:57:18 detectron2]: Rank of current process: 0. World size: 1
[11/22 15:57:19 detectron2]: Environment info:
----------------------  --------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.9.18 (main, Sep 11 2023, 13:41:44) [GCC 11.2.0]
numpy                   1.22.4
detectron2              RegionCLIP @/home/jack/Desktop/devit/detectron2
Compiler                GCC 11.3
CUDA compiler           not available
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.13.1+cu117 @/home/jack/miniconda3/envs/devit/lib/python3.9/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0                   NVIDIA GeForce 940MX (arch=5.0)
CUDA_HOME               None - invalid!
Pillow                  9.5.0
torchvision             0.14.1+cu117 @/home/jack/miniconda3/envs/devit/lib/python3.9/site-packages/torchvision
torchvision arch flags  /home/jack/miniconda3/envs/devit/l

CfgNode({'VERSION': 2, 'MODEL': CfgNode({'LOAD_PROPOSALS': False, 'MASK_ON': False, 'KEYPOINT_ON': False, 'DEVICE': 'cuda', 'META_ARCHITECTURE': 'OpenSetDetectorWithExamples', 'WEIGHTS': '', 'PIXEL_MEAN': [0.48145466, 0.4578275, 0.40821073], 'PIXEL_STD': [0.26862954, 0.26130258, 0.27577711], 'BACKBONE': CfgNode({'NAME': 'build_dino_v2_vit', 'TYPE': 'small', 'FREEZE_AT': 2}), 'FPN': CfgNode({'IN_FEATURES': [], 'OUT_CHANNELS': 256, 'NORM': '', 'FUSE_TYPE': 'sum'}), 'PROPOSAL_GENERATOR': CfgNode({'NAME': 'RPN', 'MIN_SIZE': 0}), 'ANCHOR_GENERATOR': CfgNode({'NAME': 'DefaultAnchorGenerator', 'SIZES': [[32, 64, 128, 256, 512]], 'ASPECT_RATIOS': [[0.5, 1.0, 2.0]], 'ANGLES': [[-90, 0, 90]], 'OFFSET': 0.0}), 'RPN': CfgNode({'HEAD_NAME': 'StandardRPNHead', 'IN_FEATURES': ['res4'], 'BOUNDARY_THRESH': -1, 'IOU_THRESHOLDS': [0.3, 0.7], 'IOU_LABELS': [0, -1, 1], 'BATCH_SIZE_PER_IMAGE': 256, 'POSITIVE_FRACTION': 0.5, 'BBOX_REG_LOSS_TYPE': 'smooth_l1', 'BBOX_REG_LOSS_WEIGHT': 1.0, 'BBOX_REG_WEIGHTS': 

In [3]:
# model = Trainer.build_model(cfg)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=args['resume'])

[11/22 15:57:22 d2.backbone]: Added LoRA adapters to ['fc2', 'fc1', 'proj', 'qkv']


/home/jack/Desktop/devit/detectron2/modeling/meta_arch/devit.py:453: UserWarning: label_names not found in class_prototypes_file, using COCO_SEEN_CLS + COCO_UNSEEN_CLS
  warnings.warn("label_names not found in class_prototypes_file, using COCO_SEEN_CLS + COCO_UNSEEN_CLS")


[11/22 15:57:48 d2.data.datasets.coco]: Loading datasets/coco/annotations/ovd_ins_train2017_b.json takes 24.14 seconds.
[11/22 15:57:50 d2.data.datasets.coco]: Loaded 107761 images in COCO format from datasets/coco/annotations/ovd_ins_train2017_b.json
[11/22 15:58:05 d2.data.build]: Removed 0 images with no usable annotations. 107761 images left.
[11/22 15:58:19 d2.data.build]: Distribution of instances among all 48 categories:
|  category  | #instances   |   category   | #instances   |  category  | #instances   |
|:----------:|:-------------|:------------:|:-------------|:----------:|:-------------|
|   person   | 257253       |   bicycle    | 7056         |    car     | 43533        |
| motorcycle | 8654         |    train     | 4570         |   truck    | 9970         |
|    boat    | 10576        |    bench     | 9820         |    bird    | 10542        |
|   horse    | 6567         |    sheep     | 9223         |    bear    | 1294         |
|   zebra    | 5269         |   giraffe 

In [4]:
model = trainer.model

In [5]:
# from detectron2.data import build_detection_test_loader, get_detection_dataset_dicts, DatasetCatalog, MetadataCatalog

# DatasetCatalog.get('coco_2017_ovd_all_train')

In [6]:
model.backbone

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(
      3, 384, kernel_size=(14, 14), stride=(14, 14)
      (lora_dropout): ModuleDict(
        (lora): Dropout(p=0.1, inplace=False)
      )
      (lora_A): ModuleDict(
        (lora): Conv2d(3, 64, kernel_size=(14, 14), stride=(14, 14), bias=False)
      )
      (lora_B): ModuleDict(
        (lora): Conv2d(64, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (lora_embedding_A): ParameterDict()
      (lora_embedding_B): ParameterDict()
    )
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0): NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(
          in_features=384, out_features=1152, bias=True
          (lora_dropout): ModuleDict(
            (lora): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict(
            (lora): Linear(in_features=384, out_features=64, bias=False)
   

In [7]:

params = [param for param in model.backbone.named_parameters() if param[1].requires_grad==True]
params


[('patch_embed.proj.lora_A.lora.weight',
  Parameter containing:
  tensor([[[[-3.8691e-02, -1.6043e-02, -3.9129e-03,  ..., -1.0073e-02,
              3.3424e-02, -6.1387e-03],
            [-3.9859e-02,  2.9325e-02, -1.5676e-03,  ...,  3.7490e-02,
             -2.1755e-02,  1.5274e-03],
            [ 1.6126e-02, -5.5663e-03,  1.0791e-02,  ...,  1.1159e-02,
             -3.8566e-02, -1.9341e-02],
            ...,
            [ 2.6645e-02, -1.6889e-02,  1.8070e-02,  ..., -2.9643e-02,
             -3.5910e-02, -3.7493e-02],
            [ 3.1635e-02,  1.5529e-02,  3.4659e-02,  ...,  1.0973e-02,
             -3.4070e-02, -4.0476e-02],
            [ 1.5038e-02, -1.5558e-02,  1.8659e-03,  ...,  5.3331e-03,
             -3.7572e-02,  1.6762e-02]],
  
           [[ 5.6286e-04,  1.9094e-02,  1.2440e-02,  ..., -2.5272e-02,
              3.6511e-02,  2.8280e-02],
            [ 1.0986e-02,  2.9362e-02, -2.1248e-02,  ...,  8.5338e-03,
             -2.6602e-02, -1.9349e-02],
            [-2.7719e-02, 